In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import pickle
import socket
import struct
import time
import cv2
from socket import SOCK_SEQPACKET
from queue import Queue
import threading

In [2]:
def checkMask(frame):
    
    prototxtPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\deploy.prototxt"
    weightsPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    maskNet = load_model(r"C:\Users\vasu2\Desktop\Mask Detection Test\maskdata\MaskDetectionModel")


    frame = imutils.resize(frame, width=400)
    assert not isinstance(frame,type(None)), 'frame not found'
    preds = detect_and_predict_mask(frame, faceNet, maskNet)

    
    pred = zip(preds)

    try:
        str = list(pred)[0][0]
    except:
        return 3
    
    if str[0] > str[1]:
        return 1
    else:
        return 2

In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
        
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
            
            
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
        
            
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return preds

In [4]:
HOST = '192.168.0.126'
PORT = 8089
global conn

a = "MASK!"
b = "NO MASK!"
c = "FACE NOT DETECTED!"
result = ""


def createSocket():
    global s
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print('Socket created')
    except socket.error as e:
        print("Error: " + str(e))

        
        
def socketBind():
    global s
    try:
        s.bind((HOST, PORT))
        print('Socket bind complete')
    except socket.error as e:
        print("Error: " + str(e))


        
        
def socketListen():
    global s
    global conn
    global address
    
    s.listen(10)
    conn, address = s.accept()
    print(f"Connection from {address} has been established.")




def reteriveData():
    global s
    global conn
    global address
    global data
    global frame_data
    
    
    data = b'' ### CHANGED
    payload_size = struct.calcsize("L") ### CHANGED
    # Retrieve message size
    while len(data) < payload_size:
        data += conn.recv(4096)

    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("L", packed_msg_size)[0] ### CHANGED

    # Retrieve all data based on message size
    while len(data) < msg_size:
        data += conn.recv(4096)

    frame_data = data[:msg_size]
    data = data[msg_size:]
    
    
   
        
        
    

    
def extractFrame():
    global s
    global frame_data
    global result
    # Extract frame
    t0= time.time()
    frame = pickle.loads(frame_data)
    
    t1= time.time()
    i = checkMask(frame)
    print("\nPickle Time: ", t1-t0)
    print("\nCheck Time: ", time.time()-t1,"\n\n")
    
    print (i)
    if i == 1:
        result = a
    elif i == 2:
        result = b
    elif i == 3:
        result = c
    
    

def sendReply():
    global conn
    global address
    global result
    print (result)
    conn.send(bytes(result,"utf-8"))
    conn.close()

        
def executeSystem():
    createSocket()
    socketBind()
    
    
    while True:
        socketListen()
        t0= time.time()
        reteriveData()
        
        t1= time.time()
        extractFrame()
        
        t2= time.time()
        sendReply()
        print("\nRetrieve Time: ", t1-t0)
        print("Extract Time: ", t2-t1)
        print("Reply Time: ", time.time() - t2)
        print("Total Time: ", time.time() - t0)

    #reateThreads()
    #reateJobs()

In [ ]:
executeSystem()

Socket created
Socket bind complete
Connection from ('192.168.0.163', 55040) has been established.

Pickle Time:  0.04136037826538086

Check Time:  8.069502830505371 


2
NO MASK!

Retrieve Time:  0.1613597869873047
Extract Time:  8.110863208770752
Reply Time:  0.001003265380859375
Total Time:  8.273226261138916
Connection from ('192.168.0.163', 55368) has been established.

Pickle Time:  0.0

Check Time:  1.5699801445007324 


2
NO MASK!

Retrieve Time:  0.10469198226928711
Extract Time:  1.5699801445007324
Reply Time:  0.0
Total Time:  1.6746721267700195
Connection from ('192.168.0.163', 55370) has been established.

Pickle Time:  0.0010251998901367188

Check Time:  1.4395220279693604 


2
NO MASK!

Retrieve Time:  0.1446681022644043
Extract Time:  1.440547227859497
Reply Time:  0.0
Total Time:  1.5852153301239014
Connection from ('192.168.0.163', 55372) has been established.

Pickle Time:  0.0

Check Time:  1.5245404243469238 


2
NO MASK!

Retrieve Time:  0.15400409698486328
Extrac


Pickle Time:  0.0

Check Time:  1.519763708114624 


2
NO MASK!

Retrieve Time:  0.146897554397583
Extract Time:  1.519763708114624
Reply Time:  0.0
Total Time:  1.666661262512207
Connection from ('192.168.0.163', 55390) has been established.

Pickle Time:  0.0

Check Time:  1.3849728107452393 


2
NO MASK!

Retrieve Time:  0.2153928279876709
Extract Time:  1.385969638824463
Reply Time:  0.0
Total Time:  1.6013624668121338
Connection from ('192.168.0.163', 55392) has been established.

Pickle Time:  0.0

Check Time:  1.5175344944000244 


2
NO MASK!

Retrieve Time:  0.09297657012939453
Extract Time:  1.5175344944000244
Reply Time:  0.0
Total Time:  1.610511064529419
Connection from ('192.168.0.163', 55394) has been established.

Pickle Time:  0.0

Check Time:  1.4894165992736816 


2
NO MASK!

Retrieve Time:  0.10404491424560547
Extract Time:  1.4894165992736816
Reply Time:  0.0
Total Time:  1.593461513519287
Connection from ('192.168.0.163', 55396) has been established.

Pickle Time:


Pickle Time:  0.0

Check Time:  1.665287733078003 


2
NO MASK!

Retrieve Time:  0.2611668109893799
Extract Time:  1.665287733078003
Reply Time:  0.0009968280792236328
Total Time:  1.9274513721466064
Connection from ('192.168.0.163', 55644) has been established.

Pickle Time:  0.0

Check Time:  1.4438395500183105 


2
NO MASK!

Retrieve Time:  0.16292667388916016
Extract Time:  1.4438395500183105
Reply Time:  0.0
Total Time:  1.6067662239074707
Connection from ('192.168.0.163', 55646) has been established.

Pickle Time:  0.0

Check Time:  1.5104475021362305 


2
NO MASK!

Retrieve Time:  0.11676597595214844
Extract Time:  1.5104475021362305
Reply Time:  0.0
Total Time:  1.627213478088379
Connection from ('192.168.0.163', 55648) has been established.

Pickle Time:  0.0

Check Time:  0.8278067111968994 


3
FACE NOT DETECTED!

Retrieve Time:  0.18809723854064941
Extract Time:  0.8296260833740234
Reply Time:  0.0
Total Time:  1.0177233219146729
Connection from ('192.168.0.163', 55650) ha

Connection from ('192.168.0.163', 55666) has been established.

Pickle Time:  0.0

Check Time:  1.5056710243225098 


2
NO MASK!

Retrieve Time:  0.3495032787322998
Extract Time:  1.5066680908203125
Reply Time:  0.0
Total Time:  1.8561713695526123
Connection from ('192.168.0.163', 55668) has been established.

Pickle Time:  0.0

Check Time:  1.3771708011627197 


1
MASK!

Retrieve Time:  0.14229512214660645
Extract Time:  1.3771708011627197
Reply Time:  0.0
Total Time:  1.5194659233093262
Connection from ('192.168.0.163', 55670) has been established.

Pickle Time:  0.0

Check Time:  1.5223662853240967 


2
NO MASK!

Retrieve Time:  0.15746545791625977
Extract Time:  1.5223662853240967
Reply Time:  0.0
Total Time:  1.6798317432403564
Connection from ('192.168.0.163', 55672) has been established.

Pickle Time:  0.0

Check Time:  1.4832792282104492 


2
NO MASK!

Retrieve Time:  0.16338396072387695
Extract Time:  1.4832792282104492
Reply Time:  0.000997781753540039
Total Time:  1.64766097

In [1]:
# socket -> bind -> listen -> accept


# receiveframe - > checkformask -> reply


# rpi -> connect -> framesend -> reply -> condition



# 1) time -> 30 - 40sec and sensor issue
# 2) time -> 4sec ( system was designed to check for mask twice)

In [2]:
_name_

NameError: name '_name_' is not defined

In [3]:
__name__

'__main__'

In [4]:
import sqlite3 as db
maskDB = db.connect("maskDB.db")
cur = maskDB.cursor()
cur.execute("INSERT INTO maskData(date,time,address,temp,imageLoc) VALUES (?,?,?,?,?);", ("dateTimeList[0]","dateTimeList[1]","str(address)","str(temp","loc"))
maskDB.commit()